In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [9]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [20]:
pip install bs4 #installing BeautifulSoup to transfer the data from HTML to dataframe

  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [21]:
from bs4 import BeautifulSoup
!pip install lxml

!pip install et_xmlfile

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

source = requests.get(url).text

soup = BeautifulSoup(source, 'html.parser')

table = soup.find("table")

table_rows = table.tbody.find_all("tr")

In [22]:
postalCodeList = [] #create lists to store data
boroughList = []
neighborhoodList = []

In [23]:
# for each row of the table, find all the table data
soup.find('table').find_all('tr')

In [49]:
for row in soup.find('table').find_all('tr'): #append data into lists
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [51]:
#getting rid of 'Not assigbed' in the Borough column 
toronto1_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
toronto1_df_dropna = toronto1_df[toronto1_df.Borough != "Not assigned\n"].reset_index(drop=True)
for index, row in toronto1_df_dropna.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
toronto1_df['PostalCode'] = toronto1_df['PostalCode'].str.replace(r'\n', '')
toronto1_df_dropna.head()

PostalCode             Borough                                 Neighborhood
0      M3A\n        North York\n                                    Parkwoods
1      M4A\n        North York\n                             Victoria Village
2      M5A\n  Downtown Toronto\n                    Regent Park, Harbourfront
3      M6A\n        North York\n             Lawrence Manor, Lawrence Heights
4      M7A\n  Downtown Toronto\n  Queen's Park, Ontario Provincial Government

In [53]:
#grouping of neighbourhoods 
toronto1_df_group = toronto1_df_dropna.groupby (['PostalCode','Borough'],as_index=False).agg(lambda x: ", ".join(x))
toronto1_df_group['PostalCode'] = toronto1_df_group['PostalCode'].str.replace(r'\n', '')
toronto1_df_group.head()

PostalCode        Borough                                       Neighborhood
0        M1B  Scarborough\n                     Malvern, Rouge, Malvern, Rouge
1        M1C  Scarborough\n  Rouge Hill, Port Union, Highland Creek, Rouge ...
2        M1E  Scarborough\n  Guildwood, Morningside, West Hill, Guildwood, ...
3        M1G  Scarborough\n                                     Woburn, Woburn
4        M1H  Scarborough\n                               Cedarbrae, Cedarbrae

In [34]:
# print the number of rows of the cleaned dataframe
toronto1_df_group.shape

(103, 3)

In [39]:
!wget -q -O 'customer_segmentation.csv' http://cocl.us/Geospatial_Coordinates
print('Data downloaded!')

Data downloaded!


In [45]:
toronto_data = pd.read_csv('toronto_data.csv')
toronto_data.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [54]:
#Create one table instead of two
toronto_sum = toronto1_df_group.merge(toronto_data, on="PostalCode", how="left")
toronto_sum.head()

PostalCode        Borough  \
0        M1B  Scarborough\n   
1        M1C  Scarborough\n   
2        M1E  Scarborough\n   
3        M1G  Scarborough\n   
4        M1H  Scarborough\n   

                                        Neighborhood   Latitude  Longitude  
0                     Malvern, Rouge, Malvern, Rouge  43.806686 -79.194353  
1  Rouge Hill, Port Union, Highland Creek, Rouge ...  43.784535 -79.160497  
2  Guildwood, Morningside, West Hill, Guildwood, ...  43.763573 -79.188711  
3                                     Woburn, Woburn  43.770992 -79.216917  
4                               Cedarbrae, Cedarbrae  43.773136 -79.239476